# Time series project code

In this assignment, you will write a program that plays the part of a “smart building”. This program will be
given a real-time stream of sensor data from throughout the building, and use this data to decide whether
to turn on the lights in each room. Your goal is to minimise the cost of lighting in the building, while also
trying to make sure that the lights stay on if there are people in a room. Every 15 seconds, you will receive a
new data point and have to decide whether each light should be turned on or off. There are several types
of sensors in the building, each with different reliability and data output. You will be given a file called
data.csv containing one day of complete data with all sensor values and the number of people in each room.

# Data

The file data.csv contains complete data that is representative of a typical weekday in the office building.
This data includes the output of each sensor as well as the true number of people in each room. This data
was generated using a simulation of the building, and your program will be tested against many days of data
generated by the same simulation. Because this data would be expensive to collect, you are only given 2400
complete data points, from a single workday. The simulation attempts to be a realistic approximation to
reality, so it includes many different types of noise and bias. You should treat this project as if the data came
from a real office building, and is to be tested on real data from that building. You can make any assumptions
that you think would be reasonable in the real world, and you should describe all assumptions in the report.
Part of your mark will be determined by the feasibility of your assumptions, if applied to the real world.
The number of people who come to the office each day varies according to this distribution: num_people
= round(Normal(mean=20, stddev=1)). This information was obtained from records of the number of
workers present each day, and the empirical distribution of num_people was found to be identical to
round(Normal(mean=20, stddev=1)).

# Transition matrix

In [ ]:
import pandas as pd
import numpy as np
from config import outcomeSpace

# each row - interval 15s
def prob_self_loop_and_neighbour(time_period, name, outcomeSpace):
    transition_matrix = {}
    room_list = list(outcomeSpace.keys())
    
    for src in outcomeSpace.keys():
        
        transition = [0] * 41 # initialise 41 locations with the transition matrix
        
        # NOTE: caculate the prob of moving from src to its neighbour 
        count_move_out = 0
        count_self_total = 0
        for idx in range(time_period.shape[0] - 1): # for every data
            cur_time = time_period.iloc[idx]
            next_time = time_period.iloc[idx+1]
            diff_src = next_time[src] - cur_time[src] # different in pppl for src room
            count_self_total += cur_time[src]
            
            for des in outcomeSpace[src]: # for every possiable next move to room
                diff_des = next_time[des] - cur_time[des]
                room_idx = room_list.index(des)
                
                if diff_src < 0 and diff_des > 0: # decrease in current loation ppl thus potential move to other room   
                    diff_add = abs(diff_src) if abs(diff_src) <= diff_des else diff_des # lower bound between src and des room
                    transition[room_idx] += diff_add # decrease in src, increase in des, and assume src move to des
                    count_move_out += abs(diff_src)
                if diff_src > 0 and diff_des > 0:
                    diff_add = max(abs(diff_des - diff_src), 2)
                    transition[room_idx] += diff_add
                    
                if diff_src < 0 and diff_des < 0 and diff_src < diff_des:
                    diff_add = min(abs(diff_des - diff_src), 2)
                    transition[room_idx] += diff_add
                
                    
        src_idx = room_list.index(src)
        # print(1 - count_move_out/count_self_total)
        if count_self_total != 0:
            transition /= count_self_total # normalise in terms of total ppl in that room accorss all period
            transition[src_idx] += 1 - count_move_out/count_self_total # percentage ppl stay, 1 - move out perc
        
        transition_matrix[src] = transition 
    
    out_csv = pd.DataFrame(transition_matrix)
    #out_csv = out_csv.div(out_csv.sum(axis=1), axis=0).fillna(0) # normalisation
    out_csv = np.round(out_csv, 3) # round up to save time complexity
    out_csv.to_csv(f'transition_matrix_{name}.csv',index=False)

df = pd.read_csv('data.csv')
morning_rush = df[:32] # 10 - 10*60/15, assume first 10min will be morning rush
work = df[32:-120] # in between 
home = df[-120:-72]
late_night = df[-72:]# 72 = 18*60/15

# prob_self_loop_and_neighbour(morning_rush, 'morn', outcomeSpace)
prob_self_loop_and_neighbour(work, 'work', outcomeSpace)
# prob_self_loop_and_neighbour(home, 'home', outcomeSpace)
prob_self_loop_and_neighbour(late_night, 'night', outcomeSpace)
#prob_self_loop_and_neighbour(df[:-72], 'df', outcomeSpace)

# Reliabilty

In [ ]:
"""
This file calculate the reliabilty of different sensors based on
the TP, FN rate
Note:
    * No None found in the data.csv and since no training during the testing so ignore this case
"""

import numpy as np
import pandas as pd

df = pd.read_csv('data.csv')

# two robots
def is_robot_data_true(row, rname):
    """compare the results from the robot sensors vs the actual number of ppl in the room
    data format in: ('r1', 0)

    Args:
        row (df): [description]
        rname (str): name of the robot

    Returns:
        bool: if the same
    """
    room = row[rname].split('\'')[1] 
    expected = int(row[rname].split(',')[1].split(')')[0].replace(' ', ''))
    actual = row[room]
    #print(type(expected), type(actual))
    return expected == actual

def get_robot_accuracy(df):
    """get accuracy on robot sensors

    Args:
        df (dataframe): data.csv
    """
    robot1_true_count = 0
    robot2_true_count = 0

    for i, row in df.iterrows():
        
        if is_robot_data_true(row, 'robot1'):
            robot1_true_count += 1
        if is_robot_data_true(row, 'robot2'):
            robot2_true_count += 1
    print(robot1_true_count / df.shape[0])
    print(robot2_true_count / df.shape[0])

#get_robot_accuracy(df) # 1.0, 1.0

def get_reliable_sensor_accuracy(df, sensor, room):
    """count the accuracy when data is motion and no motion 

    Args:
        df ([type]): [description]
        sensor (str): name
        room (str): name
    """
    motion = 'motion'
    no_motion = 'no motion'
    total_motion = len(df[df[sensor]==motion])
    total_station = len(df[df[sensor]==no_motion])
    count_true = len(df[(df[sensor]==motion) & (df[room]>0)])
    count_fase = len(df[(df[sensor]==no_motion) & (df[room]>0)])
    
    print('when motion: ', count_true/total_motion)
    print('when no motion: ', count_fase/total_station)
    
def get_reliable_sensor_result(df):
    reliable_sensor_dict = {
        'reliable_sensor1': 'r16', # 0.9634247714048213, 0.009181969949916527
        'reliable_sensor2': 'r5', # 0.7062146892655368, 0.0016042780748663102
        'reliable_sensor3': 'r25', # 0.9025356576862124, 0.014047410008779631
        'reliable_sensor4': 'r31' # 0.96913985554826, 0.009111617312072893
    }
    unreliable_sensor_dict = {
        'unreliable_sensor1': 'o1', # 0.8131313131313131, 0.0187192118226601
        'unreliable_sensor2': 'c3', # 0.7169373549883991, 0.0351985559566787
        'unreliable_sensor3': 'r1', # 0.7863013698630137, 0.016845329249617153
        'unreliable_sensor4': 'r24' # 0.3543307086614173, 0.0036978341257263604
    }
    for sensor, room in reliable_sensor_dict.items():
        print()
        print(sensor, room)
        get_reliable_sensor_accuracy(df, sensor, room)
        print()
        
    for sensor, room in unreliable_sensor_dict.items():
        print()
        print(sensor, room)
        get_reliable_sensor_accuracy(df, sensor, room)
        print()

#get_reliable_sensor_result(df)


# door_sensors count how many people passed through a door (in either direction), so it can be any integer.
# two rooms
def get_door_accuracy(df, sensor, room1, room2):
    
    total_motion = len(df[df[sensor]>0])
    total_station = len(df[df[sensor]==0])
    
    count_room1_true = len(df[(df[sensor]>0) & (df[room1]>0)])
    count_room1_false = len(df[(df[sensor]==0) & (df[room1]>0)])
    
    count_room2_true = len(df[(df[sensor]>0) & (df[room2]>0)])
    count_room2_false = len(df[(df[sensor]==0) & (df[room2]>0)])
    
    print('room 1')
    print('when motion: ', count_room1_true/total_motion)
    print('when station: ', count_room1_false/total_station)
    print('room2')
    print('when motion: ', count_room2_true/total_motion)
    print('when station: ', count_room2_false/total_station)

def get_door_result(df):
    door_dict = {
        'door_sensor1': ('r8', 'r9'), # 0.39090909090909093, 0.044522042776080316; 0.6090909090909091, 0.24792666957660411
        'door_sensor2': ('c1','c2'), # 0.940677966101695, 0.8438799076212471; 0.7033898305084746, 0.43648960739030024
        'door_sensor3': ('r26','r27'), # 0.5149253731343284, 0.08998676665196295; 0.6492537313432836, 0.24702249669166298
        'door_sensor4': ('r35','c4') # 0.5666666666666667, 0.15404586758978797; 0.9777777777777777, 0.8788403288619645
    }
    for sensor, room in door_dict.items():
        room1, room2 = room
        print()
        print(sensor, room1, room2)
        get_door_accuracy(df, sensor, room1, room2)
        print()
        
#get_door_result(df)

# Config

In [ ]:
"""
Below refactor is used only becaues too many variables names 
may cause typo
"""
outcomeSpace = {
    'r1':('r2','r3'),
    'r2':('r1','r4'),
    'r3':('r1','r7'),
    'r4':('r2','r8'),
    'r5':('r6','r9','c3'),
    'r6':('r5','c3'),
    'r7':('r3','c1'),
    'r8':('r4','r9'),
    'r9':('r5','r8','r13'),
    'r10':('c3',),
    'r11':('c3',),
    'r12':('outside','r22'),
    'r13':('r9','r24'),
    'r14':('r24',),
    'r15':('c3',),
    'r16':('c3',),
    'r17':('c3',),
    'r18':('c3',),
    'r19':('c3',),
    'r20':('c3',),
    'r21':('c3',),
    'r22':('r12','r25'),
    'r23':('r24',),
    'r24':('r13','r14','r23'),
    'r25':('r22','r26','c1'),
    'r26':('r25','r27'),
    'r27':('r26','r32'),
    'r28':('c4',),
    'r29':('c4','r30'),
    'r30':('r29',),
    'r31':('r32',),
    'r32':('r27','r31','r33'),
    'r33':('r32',),
    'r34':('c2',),
    'r35':('c4',),
    'c1':('r7','r25','c2'),
    'c2':('c1','r34','c4'),
    'c3': ('r5','r6','r10','r11','r15','r16','r17','r18','r19','r20','r21','o1'),
    'c4':('r29','c2','r35','r28','o1'),
    'o1': ('c3','c4'),
    'outside':('r12',) 
}

def get_idx_in_state(key):
    return list(outcomeSpace.keys()).index(key)
def is_below(value, th=0.5):
    return value < th
def is_above(value, th=0.2):
    return value > th

def set_robot_data(state, data, robot):
    """since robot accuracy is 1 and so set it directly"""
    if data[robot]:
        room = data[robot].split('\'')[1] 
        robot_data = int(data[robot].split(',')[1].split(')')[0].replace(' ', ''))
        idx = get_idx_in_state(room)
        state[idx] = int(robot_data)
        
def set_evidence_based_sensor(data, state):
    """
    set evidence based on the robot sensor and the relibilty calculated in the related file
    """
    # reliable sensors
    liable_sensor_dict = {
        'reliable_sensor1': 'r16', # 0.96, 0.01
        'reliable_sensor2': 'r5', # 0.71, 0.002
        'reliable_sensor3': 'r25', # 0.9, 0.014
        'reliable_sensor4': 'r31', # 0.97, 0.01
        'unreliable_sensor1': 'o1', # 0.81, 0.02
        'unreliable_sensor2': 'c3', # 0.72, 0.04
        'unreliable_sensor3': 'r1', # 0.79, 0.02
        'unreliable_sensor4': 'r24' # 0.35, 0.005
    }
    liable_sensor_threshold = {
        'reliable_sensor1': {'motion':(is_below, 0.96), 'no motion': (is_above, 0.01)}, 
        'reliable_sensor2': {'motion':(is_below, 0.71), 'no motion': (is_above, 0.002)}, 
        'reliable_sensor3': {'motion':(is_below, 0.9), 'no motion': (is_above, 0.014)}, 
        'reliable_sensor4': {'motion':(is_below, 0.97), 'no motion': (is_above, 0.01)}, 
        'unreliable_sensor1': {'motion':(is_below, 0.81), 'no motion': (is_above, 0.02)}, 
        'unreliable_sensor2': {'motion':(is_below, 0.72), 'no motion': (is_above, 0.04)}, 
        'unreliable_sensor3': {'motion':(is_below, 0.79), 'no motion': (is_above, 0.02)}, 
        'unreliable_sensor4': {'motion':(is_below, 0.35), 'no motion': (is_above, 0.005)}, 
    }
    
    door_dict = {
        'door_sensor1': ('r8', 'r9'), # 0.39, 0.05; 0.61, 0.25
        'door_sensor2': ('c1','c2'), # 0.94, 0.84; 0.7, 0.44
        'door_sensor3': ('r26','r27'), # 0.51, 0.1; 0.65, 0.25
        'door_sensor4': ('r35','c4') # 0.57, 0.15; 0.98, 0.88
    }
    door_threshold = {
        'door_sensor1': {'motion':(is_below, (0.39,0.05) ), 'no motion': (is_below, (0.61,0.25) )}, 
        'door_sensor2': {'motion':(is_below, (0.94,0.84) ), 'no motion': (is_below, (0.7,0.44) )},
        'door_sensor3': {'motion':(is_below, (0.51,0.1) ), 'no motion': (is_below, (0.65,0.25) )},
        'door_sensor4': {'motion':(is_below, (0.57,0.15) ), 'no motion': (is_below, (0.98,0.88) )}
    }
    # robot
    # the accuracy for the robot based on the training data is 1, so assign directly
    set_robot_data(state, data, 'robot1')
    set_robot_data(state, data, 'robot2')
    
    # liable
    for sensor, room in liable_sensor_dict.items():
        if not data[sensor]: # handle none from sensor data
            continue
        idx = get_idx_in_state(room)
        value = liable_sensor_threshold[sensor][data[sensor]] # change value depends on whether current data is less or above the threshold
        over_below_threshold = value[0](state[idx]) # e.g. is_below
        new_value = value[1] # e.g. 0.96
        if over_below_threshold: # pass value to the function - isabove/below
            state[idx] = new_value# set new value
    # door
    for sensor, rooms in door_dict.items():
        if not data[sensor]:
            continue
        room1, room2 = rooms
        idx1 = get_idx_in_state(room1)
        idx2 = get_idx_in_state(room2)
        motion = 'motion'
        if int(data[sensor]) == 0: # no flow in door, cannot be None
            motion = 'no motion'
            
        value = door_threshold[sensor][motion] # change value depends on whether current data is less or above the threshold
        over_below_threshold1 = value[0](state[idx1]) # e.g. is_below
        over_below_threshold2 = value[0](state[idx2]) # e.g. is_below
        new_value1, new_value2 = value[1] # e.g. 0.96
        
        if over_below_threshold1: # pass value to the function - isabove/below
            state[idx1] = new_value1 # set new value   
        if over_below_threshold2: # pass value to the function - isabove/below
            state[idx2] = new_value2 # set new value   
            

# Main

In [ ]:
'''
COMP9418 Assignment 2
This file is the example code to show how the assignment will be tested.

'''

# Make division default to floating-point, saving confusion
from __future__ import division
from __future__ import print_function

# Allowed libraries 
import numpy as np
import pandas as pd
import scipy as sp
import scipy.special
import heapq as pq
import matplotlib as mp
import matplotlib.pyplot as plt
import math
from itertools import product, combinations
from collections import OrderedDict as odict
import collections
from graphviz import Digraph, Graph
from tabulate import tabulate
import copy
import sys
import os
import datetime
#import sklearn
import ast
import re
import pickle
import json

from config import set_evidence_based_sensor

###################################
# Code stub
# 
# The only requirement of this file is that is must contain a function called get_action,
# and that function must take sensor_data as an argument, and return an actions_dict
# 


# this global state variable demonstrates how to keep track of information over multiple 
# calls to get_action 

# initial probability distribution
state = np.append(np.array([0 for i in range(40)]), 1) # assume starting state with 20 ppl from 'outside' location, this is observed FROM the first few seconds

params1 = pd.read_csv('transition_matrix_morn.csv')
params2 = pd.read_csv('transition_matrix_df.csv')
params3 = pd.read_csv('transition_matrix_home.csv')
params4 = pd.read_csv('transition_matrix_night.csv')
transition_matrix = {
    'morn': params1.values, 
    'work': params2.values, 
    'home': params3.values, 
    'night': params4.values
}

def get_action(sensor_data):
    """choose transition matrix based on given time period
    set evidence to HMM based on sensors
    based on threshold, turn on/off the light

    Args:
        sensor_data ([type]): [description]

    Returns:
        dict: light on/off
    """
    global state
    global transition_matrix  
    # TODO: Add code to generate your chosen actions, using the current state and sensor_data
    
    # use dfferent transition matrix depends on the peak hour
    hr = int(sensor_data['time'].hour)
    mins = int(sensor_data['time'].minute)
    
    # if hr == 8 and mins < 8: # first 8 mins
    #     state = state @ transition_matrix['morn']
    # elif hr == 17 and mins < 30: # last 18 mins
    #     state = state @ transition_matrix['work']
    # elif hr == 17 and mins < 42: # last 18 mins
    #     state = state @ transition_matrix['home']
    if hr == 17 and mins > 42:
        state = state @ transition_matrix['night']
    else:
        state = state @ transition_matrix['work']  
    set_evidence_based_sensor(sensor_data, state)
    
    # return the action dict
    actions_dict = {'lights1': 'off', 'lights2': 'on', 'lights3': 'off', 'lights4': 'off', 'lights5': 'off', 'lights6': 'off', 'lights7': 'off', 'lights8': 'off', 'lights9': 'off', 'lights10': 'off', 'lights11': 'off', 'lights12': 'off', 'lights13': 'off', 'lights14': 'off', 'lights15': 'off', 'lights16': 'off', 'lights17': 'off', 'lights18': 'off', 'lights19': 'off', 'lights20': 'off', 'lights21': 'off', 'lights22': 'off', 'lights23': 'off', 'lights24': 'off', 'lights25': 'off', 'lights26': 'off', 'lights27': 'off', 'lights28': 'off', 'lights29': 'off', 'lights30': 'off', 'lights31': 'off', 'lights32': 'off', 'lights33': 'off', 'lights34': 'off', 'lights35':'on'}
    threshold = 0.27
    for i, light in enumerate(actions_dict.keys()):
        if state[i] > threshold:
            actions_dict[light] = 'on'
        else:
            actions_dict[light] = 'off'
    return actions_dict





# 